<a href="https://colab.research.google.com/github/yasintsc99/Breast-Cancer-Diagnosis-With-Machine-Learning/blob/main/breast_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**KÜTÜPHANELERİN EKLENMESİ**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

**VERİSETİNİN İNCELENMESİ**

In [ ]:
breast_cancer_dataset = pd.read_csv("/content/sample_data/breast-cancer.csv")
breast_cancer_dataset.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
breast_cancer_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [ ]:
y = breast_cancer_dataset["diagnosis"].map({"M": 1, "B": 0})
x = breast_cancer_dataset.drop(columns=["diagnosis","id"], axis=1)

**TRAIN-TEST SPLIT**

In [ ]:
splits = {"70-30": 0.3, "75-25": 0.25, "80-20": 0.2}

**MODELLER İÇİN HİPERPARAMETRELER**

In [ ]:
param_grids = {
    "SVM": {
        "C": [0.1, 1, 10],
        "kernel": ["linear", "rbf"],
        "gamma": ["scale", "auto"]
    },
    "Random Forest": {
        "n_estimators": [100, 200, 500],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 2]
    },
    "Gradient Boosting": {
        "n_estimators": [100, 200, 500],
        "learning_rate": [0.01, 0.1, 0.2],
        "max_depth": [3, 5, 10]
    },
    "Neural Network": {
        "hidden_layer_sizes": [(50,), (100,), (50, 50)],
        "activation": ["relu", "tanh"],
        "solver": ["adam", "sgd"],
        "alpha": [0.0001, 0.001]
    }
}

**HER BİR SPLIT VE MODEL KOMBİNASYONU İÇİN ÇALIŞTIRMA**

In [ ]:
# Sonuçları saklamak için bir DataFrame oluştur
results = []
# Her bir split ve model kombinasyonu için çalıştırma
for split_name, test_size in splits.items():
  # Veri setini train-test olarak ayır
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42)
  for model_name, param_grid in param_grids.items():
        # Modeli tanımla
        if model_name == "SVM":
            model = SVC(probability=True, random_state=42)
        elif model_name == "Random Forest":
            model = RandomForestClassifier(random_state=42)
        elif model_name == "Gradient Boosting":
            model = GradientBoostingClassifier(random_state=42)
        elif model_name == "Neural Network":
            model = MLPClassifier(max_iter=1000, random_state=42)

        # GridSearchCV ile hiperparametre optimizasyonu yap
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring="f1", cv=5, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        # Tahmin yap
        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1] if hasattr(best_model, "predict_proba") else None

        # Performans metriklerini hesapla
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_prob) if y_prob is not None else None

        # Sonuçları sakla
        results.append({
            "Split": split_name,
            "Model": model_name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1,
            "ROC-AUC": roc_auc,
            "Best Params": grid_search.best_params_
        })

# Sonuçları bir DataFrame'e çevir
results_df = pd.DataFrame(results)

# Sonuçları görüntüle
results_df

,Split,Model,Accuracy,Precision,Recall,F1-Score,ROC-AUC,Best Params
0,70-30,SVM,0.964912,0.952381,0.952381,0.952381,0.993680,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}"
1,70-30,Random Forest,0.970760,0.983333,0.936508,0.959350,0.997061,"{'max_depth': None, 'min_samples_leaf': 2, 'mi..."
2,70-30,Gradient Boosting,0.959064,0.951613,0.936508,0.944000,0.994121,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti..."
3,70-30,Neural Network,0.970760,0.983333,0.936508,0.959350,0.997501,"{'activation': 'tanh', 'alpha': 0.001, 'hidden..."
4,75-25,SVM,0.965035,0.945455,0.962963,0.954128,0.992093,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}"
5,75-25,Random Forest,0.965035,0.962264,0.944444,0.953271,0.996463,"{'max_depth': None, 'min_samples_leaf': 1, 'mi..."
6,75-25,Gradient Boosting,0.958042,0.944444,0.944444,0.944444,0.994174,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti..."
7,75-25,Neural Network,0.965035,0.980392,0.925926,0.952381,0.992301,"{'activation': 'tanh', 'alpha': 0.001, 'hidden..."
8,80-20,SVM,0.956140,0.975000,0.906977,0.939759,0.995742,"{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}"
9,80-20,Random Forest,0.964912,0.975610,0.930233,0.952381,0.995906,"{'max_depth': None, 'min_samples_leaf': 1, 'mi..."
